In [1]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import gc
from prj.config import DATA_DIR


BASE_PATH = DATA_DIR


partition=9
train_ds = pl.concat([
    pl.read_parquet(BASE_PATH / f'partition_id={i}' / 'part-0.parquet')
    for i in range(partition, partition + 1)
]).sort('date_id', 'time_id', 'symbol_id')
features = [col for col in train_ds.columns if col.startswith('feature_')]
target_feature = 'responder_6'

In [2]:
from prj.model.gbdt.EnsembleTree import EnsembleTree

agent_dicts = [
    {'agent_type': 'lgbm', 'load_path': "/home/lorecampa/projects/jane_street_forecasting/experiments/saved/LGBMRegressor_1seeds_4_5-6_6_134802/best_trial/saved_model"},
    {'agent_type': 'lgbm', 'load_path': "/home/lorecampa/projects/jane_street_forecasting/experiments/saved/LGBMRegressor_1seeds_5_6-7_7_134811/best_trial/saved_model"},
    {'agent_type': 'lgbm', 'load_path': "/home/lorecampa/projects/jane_street_forecasting/experiments/saved/LGBMRegressor_1seeds_6_7-8_8_134900/best_trial/saved_model"}
]

model = EnsembleTree(agent_dicts)
model

2024-12-15 22:05:09.797358: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 22:05:09.797421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 22:05:09.798881: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 22:05:09.806735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 22:05:10.670083: W tensorflow/compiler/tf2

Loading models, overwriting seeds: [1879224255]
Loading models, overwriting seeds: [2496896330]
Loading models, overwriting seeds: [3489235709]


In [3]:
X = train_ds[features].cast(pl.Float32).to_numpy()
y = train_ds[target_feature].to_numpy()
w = train_ds['weight'].to_numpy()

X_train = X[:int(0.8 * len(X))]
y_train = y[:int(0.8 * len(y))]
w_train = w[:int(0.8 * len(w))]

X_val = X[int(0.8 * len(X)):]
y_val = y[int(0.8 * len(y)):]
w_val = w[int(0.8 * len(w)):]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((5019660, 79), (5019660,), (1254916, 79), (1254916,))

In [4]:
model.fit(X_train, y_train, sample_weight=w_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.104578 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19324
[LightGBM] [Info] Number of data points in the train set: 5019660, number of used features: 79
[LightGBM] [Info] Start training from score -0.007563


In [5]:
model.predict(X_val)

ValueError: Number of features of the model must match the input. Model n_features_ is 79 and input n_features is 82